In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')
        

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
# Take the correct pedestal file (usually located in /sf/alvra/data/*pgroup*/res/JF_pedestals)
# Take the correct gains.h5 file (usually located in /sf/alvra/config/jungfrau/gainMaps/JF02T09V01/)

pede_file = "/sf/alvra/data/p17983/res/JF_pedestals/pedestal_20190723_1255.JF02T09V02.res.h5"

gain_file = "/sf/alvra/config/jungfrau/gainMaps/JF02T09V02/gains.h5"

with h5py.File(gain_file, "r") as file: 
    G = file["gains"][:]
    
with h5py.File(pede_file, "r") as file: 
    P = file["gains"][:]
    mask = file["pixel_mask"][:]

print("Dimensions of G: ", G.shape)
print("Dimensions of P: ", P.shape)
print("Dimensions of mask: ", mask.shape)

# Detector name for loading module map and applying geometry
detector_name = "JF02T09V02"

Dimensions of G:  (4, 4608, 1024)
Dimensions of P:  (4, 4608, 1024)
Dimensions of mask:  (4608, 1024)


In [4]:
DIR = "/sf/alvra/data/p17983/raw/scan_data/Ru_foil_monoscan_001/"
# DIR = "/sf/alvra/data/p17983/raw/"

listfile = os.listdir(DIR)
number_files = len(listfile)
print ("There are",number_files,"files in the folder", DIR)

There are 84 files in the folder /sf/alvra/data/p17983/raw/scan_data/Ru_foil_monoscan_001/


In [5]:
numberSteps = 20
roi=[200,320,4850,5250]

In [6]:
# filename = "Ru_foil_monoscan_001_step0020.JF02T09V02.h5"
# bsreadfile = "Ru_foil_monoscan_001_step0020.BSREAD.h5"

# for n=1, NumberSteps:
filename_root = "Ru_foil_monoscan_001_step0007"
# filename_root = "Ru_foil_XES_3keV_002"


filename = filename_root+".JF02T09V02.h5"
bsreadfile = filename_root+".BSREAD.h5"

print(filename)
print(bsreadfile)
#user sets number of frames to combine
N_shots_to_use = 10

images, pulse_ids = load_JF_data_corrected(DIR + filename, G, P, mask, nshots=N_shots_to_use)

#this function assumes pump on/off handeling and we just forced everything into one of them and read it out
(DataFluo, _, IzeroFEL, _, BS_pulse_ids, _, _, _) = load_PumpProbe_events(DIR+bsreadfile, channel_BS_pulse_ids,nshots=N_shots_to_use*2)    
        
nframes = images.shape[0]
print('number of frames')
print(nframes)
# nframes = 1

X = [a==b for a,b in zip(pulse_ids, BS_pulse_ids)]
if sum([int(x) for x in X]) == images.shape[0]:
    print('passed pulse id check')
else:
    print('failed pulse id check')
# print(images.shape)

numstds = 2 #2
minIzero = 0.002 #0.002
image_threshold = 1
hot_pixel = 15

total = 0
# total_off = 0

IzeroMedian = np.median(IzeroFEL)
IzeroSTD = np.std(IzeroFEL)

# print(images_off.shape)
print('number of frames')
print(nframes)
    
conditionMax = IzeroFEL < IzeroMedian+numstds*IzeroSTD
conditionMin = IzeroFEL > IzeroMedian-numstds*IzeroSTD
conditionLow = IzeroFEL > minIzero
condition = np.logical_and.reduce((conditionLow, conditionMin, conditionMax)).T[0]
# if conditionMax and conditionMin and conditionLow:

# print(condition)

images_good = images[condition,...]
#         image_off = ju.apply_gain_pede(image_off, G=G, P=P, pixel_mask=mask, highgain=False)
images_thr = images_good.copy()
images_thr[images_good < image_threshold] = 0
images_thr[images_good > hot_pixel] = 0

print('number of surviving frames')
print(images_thr.shape[0])

total = images_thr.sum(axis=0)
total = total.ravel()

print(total.shape)
    

Ru_foil_monoscan_001_step0007.JF02T09V02.h5
Ru_foil_monoscan_001_step0007.BSREAD.h5
All modules are active


/sf/alvra/anaconda/dev/lib/python3.6/site-packages/jungfrau_utils/corrections.py:202: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  res = np.stack(partial_func_to_use(i) for i in image)


number of frames
10
passed pulse id check
number of frames
10
number of surviving frames
7
(4718592,)


In [16]:
for n in range(1, numberSteps+1):
    if n < 10:
        filename_root1 = "Ru_foil_monoscan_001_step000"
    if n > 10:
        filename_root1 = "Ru_foil_monoscan_001_step00"
# filename_root = "Ru_foil_XES_3keV_002"
    print(n)
    
    
    filename = filename_root1+str(n)+".JF02T09V02.h5"
    bsreadfile = filename_root1+str(n)+".BSREAD.h5"
    
    print(filename)

1
Ru_foil_monoscan_001_step0001.JF02T09V02.h5
2
Ru_foil_monoscan_001_step0002.JF02T09V02.h5
3
Ru_foil_monoscan_001_step0003.JF02T09V02.h5
4
Ru_foil_monoscan_001_step0004.JF02T09V02.h5
5
Ru_foil_monoscan_001_step0005.JF02T09V02.h5
6
Ru_foil_monoscan_001_step0006.JF02T09V02.h5
7
Ru_foil_monoscan_001_step0007.JF02T09V02.h5
8
Ru_foil_monoscan_001_step0008.JF02T09V02.h5
9
Ru_foil_monoscan_001_step0009.JF02T09V02.h5
10
Ru_foil_monoscan_001_step00010.JF02T09V02.h5
11
Ru_foil_monoscan_001_step0011.JF02T09V02.h5
12
Ru_foil_monoscan_001_step0012.JF02T09V02.h5
13
Ru_foil_monoscan_001_step0013.JF02T09V02.h5
14
Ru_foil_monoscan_001_step0014.JF02T09V02.h5
15
Ru_foil_monoscan_001_step0015.JF02T09V02.h5
16
Ru_foil_monoscan_001_step0016.JF02T09V02.h5
17
Ru_foil_monoscan_001_step0017.JF02T09V02.h5
18
Ru_foil_monoscan_001_step0018.JF02T09V02.h5
19
Ru_foil_monoscan_001_step0019.JF02T09V02.h5
20
Ru_foil_monoscan_001_step0020.JF02T09V02.h5


In [ ]:
for 1 to numberSteps